In [7]:
import os
from pathlib import Path
import random
from monai.transforms import (
    Compose,
    LoadImaged,
    EnsureChannelFirstd,
    ScaleIntensityd
)
from monai.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patches as mpatches


# Define the path to the data directory
root_directory = Path().resolve().parent                            # Main project directory
data_directory = root_directory / 'data' / 'raw' / 'training_data'  # Data directory

# List all subfolders in the data directory
folders = [f for f in data_directory.iterdir() if f.is_dir()]

# Select the random folder
bool_random = False
first_folder = folders[random.randint(0, len(folders)-1)if bool_random else 0]

# Define the paths to the image and mask files
image_files = {
    't1c': str(first_folder / f'{first_folder.name}-t1c.nii.gz'),
    't1n': str(first_folder / f'{first_folder.name}-t1n.nii.gz'),
    't2f': str(first_folder / f'{first_folder.name}-t2f.nii.gz'),
    't2w': str(first_folder / f'{first_folder.name}-t2w.nii.gz'),
    'seg': str(first_folder / f'{first_folder.name}-seg.nii.gz')
}

# Create a list of files to process
files = [{'t1c': image_files['t1c'], 't1n': image_files['t1n'], 't2f': image_files['t2f'], 't2w': image_files['t2w'], 'label': image_files['seg']}]

# Define the transformations
transforms = Compose([
    LoadImaged(keys=['t1c', 't1n', 't2f', 't2w', 'label']),
    EnsureChannelFirstd(keys=['t1c', 't1n', 't2f', 't2w', 'label']),
    ScaleIntensityd(keys=['t1c', 't1n', 't2f', 't2w'])
])

# Create a dataset and dataloader
dataset = Dataset(data=files, transform=transforms)
loader = DataLoader(dataset, batch_size=1)

# Load a sample batch
batch = next(iter(loader))

# Extract image and label data
images = {key: batch[key][0][0] for key in ['t1c', 't1n', 't2f', 't2w']}
label = batch['label'][0][0]

# Define the color map for labels
label_colors = {
    0: [0, 0, 0],         # Background
    1: [255, 0, 0],       # Non-enhancing tumor core (NETC) - Red
    2: [0, 255, 0],       # Surrounding non-enhancing FLAIR hyperintensity (SNFH) - Green
    3: [0, 0, 255],       # Enhancing tissue (ET) - Blue
    4: [255, 255, 0]      # Resection cavity (RC) - Yellow
}

# Labels description
label_descriptions = {
    0: 'Background',
    1: 'Non-enhancing tumor core (NETC)',
    2: 'Surrounding non-enhancing FLAIR hyperintensity (SNFH)',
    3: 'Enhancing tissue (ET)',
    4: 'Resection cavity (RC)'
}

In [ ]:


# Function to color the label
def color_label(label, colors):
    colored_label = np.zeros((*label.shape, 3), dtype=np.uint8)
    for lbl, color in colors.items():
        colored_label[label == lbl] = color
    return colored_label

# Function to plot images with a colored label mask
def plot_slices(slice_idx, images, label):
    fig, axs = plt.subplots(1, 5, figsize=(20, 5))
    modalities = ['t1c', 't1n', 't2f', 't2w']
    titles = ['T1c', 'T1n', 'T2f', 'T2w', 'Mask']
    
    for i, modality in enumerate(modalities):
        axs[i].imshow(images[modality][:, :, slice_idx], cmap='gray', origin='lower')
        axs[i].set_title(titles[i])
        axs[i].axis('off')

    colored_label = color_label(label[:, :, slice_idx], label_colors)
    axs[4].imshow(images['t1c'][:, :, slice_idx], cmap='gray', origin='lower')
    axs[4].imshow(colored_label, alpha=0.5, origin='lower')
    axs[4].set_title(titles[4])
    axs[4].axis('off')

    # Add a legend
    handles = [mpatches.Patch(color=np.array(color)/255, label=f"{lbl} - {label_descriptions[lbl]}") for lbl, color in label_colors.items() if lbl != 0]
    fig.legend(handles=handles, loc='center left', bbox_to_anchor=(1, 0.5), title="Labels")
    
    plt.suptitle(f"Slice Index: {slice_idx}")
    plt.show()

for i in range(50, images['t1c'].shape[2]-40, 10):
    plot_slices(i, images, label)


In [10]:
from monai.transforms import LoadImage
# this takes 10 minutes to run

# Function to count the number of pixels for each label in a segmentation file
def count_labels(segmentation):
    unique, counts = np.unique(segmentation, return_counts=True)
    return dict(zip(unique, counts))

# Aggregate label counts from all patients
total_counts = {label: 0 for label in label_descriptions.keys()}

# Iterate through each folder and aggregate the counts for each label
for folder in folders:
    label_file = str(folder / f'{folder.name}-seg.nii.gz')
    if os.path.exists(label_file):
        # Load the label image
        label_data = LoadImage(image_only=True)(label_file)
        label_counts = count_labels(label_data)
        for label, count in label_counts.items():
            total_counts[label] += count

# Calculate the total number of pixels across all labels
total_pixels = sum(total_counts.values())

# Print the counts and percentages for each label
print("Label - Counts - % of All")
for label, count in total_counts.items():
    percentage = (count / total_pixels) * 100
    print(f"{label_descriptions[label]} - {count} - {percentage:.2f}%")


Label - Counts - % of All
Background - 9649363112 - 98.98%
Non-enhancing tumor core (NETC) - 2285704 - 0.02%
Surrounding non-enhancing FLAIR hyperintensity (SNFH) - 66476419 - 0.68%
Enhancing tissue (ET) - 11442955 - 0.12%
Resection cavity (RC) - 18825010 - 0.19%


In [13]:
import torch

# Calculate weights, excluding the background (label 0)
total_non_background_pixels = total_pixels - total_counts[0]
class_weights = {label: total_non_background_pixels / count for label, count in total_counts.items() if label != 0}

# Normalize weights
total_weight = sum(class_weights.values())
normalized_class_weights = {label: weight / total_weight for label, weight in class_weights.items()}

print("\nClass Weights:")
for label, weight in normalized_class_weights.items():
    print(f"{label_descriptions[label]} - {weight:.4f}")

# Convert class weights to a PyTorch tensor
class_weights_tensor = torch.tensor([normalized_class_weights[label] for label in sorted(normalized_class_weights.keys())], dtype=torch.float32)

print("\nClass Weights as PyTorch Tensor:")
print(class_weights_tensor)



Class Weights:
Non-enhancing tumor core (NETC) - 0.7377
Surrounding non-enhancing FLAIR hyperintensity (SNFH) - 0.0254
Enhancing tissue (ET) - 0.1474
Resection cavity (RC) - 0.0896

Class Weights as PyTorch Tensor:
tensor([0.7377, 0.0254, 0.1474, 0.0896])


In [ ]:
from monai.transforms import LoadImage

loader = LoadImage(image_only=True)

# Function to get the shape and pixel value statistics of a NIfTI image using MONAI
def get_image_stats(file_path):
    image = loader(file_path)
    shape = image.shape
    min_val = image.min()
    max_val = image.max()
    mean_val = image.mean()
    std_val = image.std()
    return shape, min_val, max_val, mean_val, std_val

# Function to check the stats of images in each folder
def check_folder_images(folder):
    image_files = {
        't1c': str(folder / f'{folder.name}-t1c.nii.gz'),
        't1n': str(folder / f'{folder.name}-t1n.nii.gz'),
        't2f': str(folder / f'{folder.name}-t2f.nii.gz'),
        't2w': str(folder / f'{folder.name}-t2w.nii.gz'),
        'seg': str(folder / f'{folder.name}-seg.nii.gz')
    }

    stats = {key: get_image_stats(file) for key, file in image_files.items() if Path(file).exists()}
    return stats

# Iterate through each folder and print the stats of the image files
for folder in folders[:5]:
    print(f"\nPatient ID: {folder.name}")
    folder_stats = check_folder_images(folder)
    for image_type, (shape, min_val, max_val, mean_val, std_val) in folder_stats.items():
        print(f"  {image_type} shape: {shape}, min: {min_val}, max: {max_val}, mean: {mean_val}, std: {std_val}")


Patient ID: BraTS-GLI-00005-100
  t1c shape: torch.Size([182, 218, 182]), min: 0.0, max: 8183.82373046875, mean: 357.1735534667969, std: 825.226806640625
  t1n shape: torch.Size([182, 218, 182]), min: 0.0, max: 4300.9189453125, mean: 293.4800109863281, std: 668.3680419921875
  t2f shape: torch.Size([182, 218, 182]), min: 0.0, max: 1406.898681640625, mean: 80.09017944335938, std: 189.46826171875
  t2w shape: torch.Size([182, 218, 182]), min: 0.0, max: 2519.352783203125, mean: 121.66287231445312, std: 307.92449951171875
  seg shape: torch.Size([182, 218, 182]), min: 0.0, max: 4.0, mean: 0.008711219765245914, std: 0.1484944224357605

Patient ID: BraTS-GLI-00005-101
  t1c shape: torch.Size([182, 218, 182]), min: 0.0, max: 7013.974609375, mean: 331.47259521484375, std: 767.3403930664062
  t1n shape: torch.Size([182, 218, 182]), min: 0.0, max: 4595.9208984375, mean: 288.7283630371094, std: 665.3131103515625
  t2f shape: torch.Size([182, 218, 182]), min: 0.0, max: 1222.2093505859375, mean: 7